<a href="https://colab.research.google.com/github/rghosh1353/spotify_genre_proj/blob/cchen03/Check_in_3_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install scikit-lego

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.9 MB/s eta 0:00:00


In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklego.linear_model import LADRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error, r2_score

In [66]:
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

In [67]:
# Data Cleaning Function
def clean_data(df):
  clean_df = df.copy()
  clean_df = clean_df.drop('Unnamed: 0', axis=1, errors='ignore')
  clean_df = clean_df.dropna()
  clean_df['explicit'] = clean_df['explicit'].astype(int)
  clean_df = clean_df[['track_id', 'artists', 'album_name', 'track_name', 'energy','explicit']]
  return clean_df

# Apply the function to the DataFrame
spotify_clean = clean_data(df)


In [68]:
# Split data into train and test
from sklearn.model_selection import train_test_split
train_spotify, val_spotify = train_test_split(spotify_clean, test_size=0.2, random_state=42)

# Stage 5: Logistic Regression

We're using


*   Predictor X: Energy
*   Response y: Explicit



In [69]:
#class_weights = {0: 5, 1: 1}  # Example: Giving class 1 five times the weight of class 0

# Compute a Logistic Regression fit for the training dataset.
lr_train = LogisticRegression(solver='liblinear', class_weight='balanced')
#lr_train = LogisticRegression(solver='liblinear'
lr_train.fit(X=train_spotify[['energy']], y=train_spotify['explicit'].values)

# View the model intercept and coefficients
lr_train.intercept_, lr_train.coef_




(array([-1.190757]), array([[1.75317216]]))

In [70]:
val_spotify.sample(10)

,track_id,artists,album_name,track_name,energy,explicit
5035,69mhZKG0nDbSK7NoINWEsE,ENHYPEN,I Need The Light,I Need The Light,0.869,0
102916,6MY1xABxoZATrEr4FeUhEM,Damien Rice,O,Delicate,0.144,0
12396,6RVBDN9xXQ6rl4S0woH33O,Joey Yung,Hopelessly Romantic Collection,連續劇,0.515,0
53640,2WKBEpRgLDXonyicWfyzdh,Mabel,pov: it's 2020,Boyfriend,0.847,0
3029,2cwOhw3d0uIsR4krTsrLwF,The Killers;Ryan Pardey,Rock n Roll Christmas,Don't Shoot Me Santa,0.847,0
30282,3GqdU7BqLVpP7hBtlncv3n,Swedish House Mafia,Larga Vida al Electro,One - Radio Edit,0.780,0
25417,0cXFXgFns7sIMmTpqyGRmD,Olivia Newton-John,Xanadu - Original Motion Picture Soundtrack,Xanadu,0.584,0
82785,26LtMtNSAFwBMl6ehOq37r,Joe Jackson,Night And Day,Real Men,0.500,0
87855,0KM3fzzUBVW0vW7RmFh28v,Connie Francis,The Collection,Stupid Cupid,0.725,0
88574,4tS4bHAZvTJdEl3jFoX08z,Alejo;Feid;ROBI,Reggaeton Mix Perreito,Pantysito,0.739,1


In [71]:
print(spotify_clean['explicit'].value_counts())


explicit
0    104252
1      9747
Name: count, dtype: int64


In [72]:
sample_index = [18726, 28097, 29054, 32086, 32670, 32366,
                33555, 42819, 47008, 51460, 59311, 60383,
                60877, 61000, 61024, 68154, 70999, 71790,
                72713, 89458, 90417, 93829, 104531, 113186,]  # Sample indices with both 0 and 1 for explicit
spotify_val_sample = val_spotify.loc[sample_index,:]  # Subset the validation data using the sample indices

# Proceed with the prediction
pred_spotify_sample = pd.DataFrame(dict(
    explicit = spotify_val_sample['explicit'],
    lr_predict = lr_train.predict_proba(spotify_val_sample[['energy']])[:, 1],  # Probabilities for 'explicit' class (1)
    lr_predict_binary = lr_train.predict(spotify_val_sample[['energy']])  # Binary predictions (0 or 1)
))

# Display the observed vs predicted table
pred_spotify_sample


,explicit,lr_predict,lr_predict_binary
18726,1,0.584133,1
28097,0,0.595162,1
29054,1,0.630909,1
32086,0,0.485890,0
32670,0,0.489394,0
32366,0,0.628047,1
33555,1,0.471891,0
42819,0,0.635794,1
47008,1,0.617338,1
51460,0,0.440154,0


In [73]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(spotify_val_sample['explicit'], pred_spotify_sample['lr_predict_binary'])
percentage = accuracy * 100

print(f"Percentage of correct predictions: {percentage:.2f}%")

Percentage of correct predictions: 62.50%


In [74]:
threshold = 0.5  # Setting the threshold to 0.3
pred_spotify_sample['lr_predict_binary_custom'] = (pred_spotify_sample['lr_predict'] >= threshold).astype(int)
pred_spotify_sample[['explicit', 'lr_predict_binary', 'lr_predict_binary_custom']]  # Displaying the comparison

,explicit,lr_predict_binary,lr_predict_binary_custom
18726,1,1,1
28097,0,1,1
29054,1,1,1
32086,0,0,0
32670,0,0,0
32366,0,1,1
33555,1,0,0
42819,0,1,1
47008,1,1,1
51460,0,0,0


In [75]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(spotify_val_sample['explicit'], pred_spotify_sample['lr_predict_binary_custom'])
percentage = accuracy * 100

print(f"Percentage of correct predictions: {percentage:.2f}%")

Percentage of correct predictions: 62.50%


In [76]:
# Create Confusion Matrix
conf_lr = confusion_matrix(y_true=pred_spotify_sample['explicit'], y_pred=pred_spotify_sample['lr_predict_binary'])
conf_lr

array([[7, 7],
       [2, 8]])

In [ ]:
# Prediction Accuracy Assessment

In [ ]:
# Sensitivity (TPR) & Specificity (TNR)

In [ ]:
# Predicted Probability Densities

In [ ]:
# ROC Curve

In [ ]:
# Area Under the Curve (under the ROC curve) Score

In [ ]:
# Cross Validation for Model Performance